# AI 4 StoryLines - Bayesian LR

In this example, we are going to showcase how we can do sensitivity analysis using a simple linear regression model.

In [1]:
import autoroot
import numpy as np
import numpyro
from numpyro.diagnostics import hpdi
import numpyro.distributions as dist
from numpyro.infer import Predictive, SVI, Trace_ELBO
from numpyro.infer.autoguide import AutoDelta
from numpyro import handlers
from numpyro.infer import MCMC, NUTS
import jax
import jax.random as jrandom
import jax.numpy as jnp
from jaxtyping import Array, Float
from pathlib import Path
from dataclasses import dataclass
import xarray as xr
import pandas as pd
import einops
import matplotlib.pyplot as plt
import seaborn as sns
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from bayesevt._src.utils.io import get_list_filenames
# from utils import plot_weights


numpyro.set_platform("cpu")
numpyro.set_host_device_count(64)
sns.reset_defaults()
sns.set_context(context="talk", font_scale=0.7)
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
rng_key = jrandom.PRNGKey(123)

We can see that there is some explained variance that is missing.
Potentially we can explain this with the regression model.

## Load Data

We have a clean analysis-ready dataset available from the previous notebook.
We will save it and revisit it later.

First, we will make sure that the models in the covariates and the QoI are the same.

In [3]:
save_dir = "/pool/usuarios/juanjohn/data/ai4storylines/analysis/"

In [4]:
# load covariates
df = pd.read_csv(Path(save_dir).joinpath("covariates.csv"), index_col=0)
# load qoi
ds = xr.open_dataset(Path(save_dir).joinpath("qoi.nc"))
# quick check
assert df.model.values.sort() == ds.model.values.sort()
ds

<xarray.Dataset> Size: 57kB
Dimensions:    (model: 28, spatial: 399)
Coordinates:
  * model      (model) <U16 2kB 'access_cm2' 'access_esm1_5' ... 'ukesm1_0_ll'
    land_mask  (spatial) int16 798B 0 0 0 1 1 1 1 1 1 1 ... 0 0 0 0 0 0 0 0 0 0
  * spatial    (spatial) object 3kB MultiIndex
  * lat        (spatial) float64 3kB 26.25 26.25 26.25 ... 71.25 71.25 71.25
  * lon        (spatial) float64 3kB -20.0 -17.5 -15.0 -12.5 ... 25.0 27.5 30.0
Data variables:
    tasmax     (model, spatial) float32 45kB 3.244 2.818 2.714 ... 8.945 8.988
Attributes: (12/49)
    CDI:                    Climate Data Interface version 2.0.5 (https://mpi...
    source:                 ACCESS-CM2 (2019): \naerosol: UKCA-GLOMAP-mode\na...
    institution:            CSIRO (Commonwealth Scientific and Industrial Res...
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            ScenarioMIP
    branch_method:          standard
    ...                     ...
    variant_label:          r1i1p1f1
    version:                v20191108
    cmor_version:           3.4.0
    tracking_id:            hdl:21.14100/9ecf99bf-2241-4c6a-acea-15d12be52c43
    license:                CMIP6 model data produced by CSIRO is licensed un...
    CDO:                    Climate Data Operators version 2.0.5 (https://mpi...

## Baseline Model - Linear Regression

#### Input Data

In [5]:
covariate_names = ["sst", "sm_sur", "t2m", "z500_zonal"]
qoi_names = ["tasmax"]
x = df.sort_values(by=["model"])[covariate_names].values
u = ds.sortby("model").tasmax..stack(spatial=["lat", "lon"]).values

In [6]:
x.shape, u.shape

((28, 4), (28, 399))

#### Model

In [7]:
num_spatial = u.shape[1]
num_models = u.shape[0]
num_covariates = x.shape[1]

In [8]:
from models import model_bhm, ModelPredictorMCMC

In [9]:
rng_key, rng_prior = jrandom.split(rng_key, 2)
prior_predictive = Predictive(model_bhm, num_samples=1000)
prior_samples = prior_predictive(rng_prior, x=x, num_spatial=num_spatial)

### Sampling: MCMC

In [10]:
# initialize kernel
nuts_kernel = NUTS(model_bhm)

# initial mcmc scheme
num_warmup = 500
num_samples = 2_000
num_chains = 5
mcmc = MCMC(nuts_kernel, num_warmup=num_warmup, num_samples=num_samples, num_chains=num_chains, chain_method="parallel")

In [11]:
%%time
rng_key, rng_mcmc = jrandom.split(rng_key, 2)

mcmc.run(rng_key=rng_mcmc, x=x, y=u, num_spatial=num_spatial)

  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/2500 [00:00<?, ?it/s]

CPU times: user 2min 25s, sys: 1.93 s, total: 2min 27s
Wall time: 44.2 s


### Posterior Samples

In [22]:
rng_key, rng_predict = jrandom.split(rng_key)

In [23]:
posterior_predictive = Predictive(model_bhm, posterior_samples=mcmc.get_samples(), parallel=True)
posterior_samples = posterior_predictive(rng_predict, x=x, num_spatial=num_spatial)

### Predictions

In [14]:
model_inference = ModelPredictorMCMC(model=model_bhm, posterior_params=mcmc.get_samples(), num_spatial=num_spatial)

$$
\begin{aligned}
\text{Sample Posterior Parameters}: && &&
\boldsymbol{\theta}_n &\sim p(\boldsymbol{\theta}|\mathcal{D}) \\
\text{Sample Covariates}: && &&
\mathbf{x}_m &\sim p(\mathbf{x}) \\
\text{Data Likelihood}: && &&
\mathbf{u}_{nm} &= p(\mathbf{u}_{nm}|\mathbf{z}_{nm})
p(\mathbf{z}_{nm}|\mathbf{x}_m,\boldsymbol{\theta}_n)
\end{aligned}
$$

In [15]:
# pred = model_inference.predict(x, rng_key=jrandom.PRNGKey(10))
# pred.shape

It can be faster to predict using vectorized operations

In [16]:
pred = jax.vmap(model_inference.predict, in_axes=(0,None), out_axes=1)(x, rng_predict).squeeze()
pred.shape

(10000, 28, 399)

### Gradients

$$
\begin{aligned}
\text{Sample Posterior Parameters}: && &&
\boldsymbol{\theta}_n &\sim p(\boldsymbol{\theta}|\mathcal{D}) \\
\text{Sample Covariates}: && &&
\mathbf{x}_m &\sim p(\mathbf{x}) \\
\text{Data Likelihood (Gradient)}: && &&
\partial_{\mathbf{x}_m}\mathbf{u}_{nm} &= p(\mathbf{u}_{nm}|\mathbf{z}_{nm})
p(\mathbf{z}_{nm}|\mathbf{x}_m,\boldsymbol{\theta}_n)
\end{aligned}
$$

In [17]:
# grads = model_inference.gradient(x, jrandom.PRNGKey(10))
# grads.shape

In [18]:
grads = jax.vmap(model_inference.gradient, in_axes=(0,None), out_axes=2)(x, rng_predict)
grads.shape

(10000, 399, 28, 4)

In [19]:
import arviz as az
import numpy as np

mcmc_arviz = az.from_numpyro(
    posterior=mcmc, 
    coords = {"chain": np.arange(0, num_chains),
              "draw": np.arange(0, num_samples),
              # "spatial": pd.MultiIndex.from_arrays([lat, lon], names=["lat", "lon"]),
              "covariate": ["sst", "sm", "t2m", "z500"],
              "model": ds.model,
             },
    dims = {
        "bias": ["chain", "draw", "spatial"],
        "weight": ["chain", "draw", "spatial", "covariate"],
        "z_scale": ["chain", "draw", "spatial"],
        "obs": ["model", "spatial"],
        
    },
    posterior_predictive=posterior_samples,
    predictions={"grads": grads, "pred": pred},
    pred_dims = {
        "grads": ["spatial", "model", "covariate"],
        "pred": ["model", "spatial"], 
    },
)


# assign the lat-lon coordinates to data structure
mcmc_arviz = mcmc_arviz.assign_coords({
    "spatial": ds[["lat", "lon"]].spatial,
    "lon": ds[["lat", "lon"]].lon,
    "lat": ds[["lat", "lon"]].lat,
})

mcmc_arviz

Inference data with groups:
	> posterior
	> posterior_predictive
	> predictions
	> log_likelihood
	> sample_stats
	> observed_data

In [20]:
results_save_dir = "/pool/usuarios/juanjohn/data/ai4storylines/results/"

In [21]:
mcmc_arviz.unstack().to_netcdf(Path(results_save_dir).joinpath("bhm_mcmc.nc"), engine="netcdf4")

PosixPath('/pool/usuarios/juanjohn/data/ai4storylines/results/bhm_mcmc.nc')